In [1]:
import numpy as np
import ctypes as ct
import sys

In [12]:
DIM = 3  # Dimensionality

class TreeNode(ct.Structure):
    pass

TreeNode._fields_ = [
        ("key", ct.c_int),
        ("prob", ct.c_float),
        ("v", ct.c_float * DIM),
        ("u", ct.c_float * DIM),
        ("w", ct.c_float * DIM),
        ("ctu", ct.c_float * DIM),
        ("state", ct.c_int * DIM),
        ("i_nodes", ct.POINTER(TreeNode) * DIM),
        ("k_nodes", ct.POINTER(TreeNode) * DIM),
        ("dcu", ct.c_float),
        ("cfl_dt", ct.c_float),
        ("new_f", ct.c_int),
        ("ik_f", ct.c_int),
        ("del_f", ct.c_int),
        ("left", ct.POINTER(TreeNode)),
        ("right", ct.POINTER(TreeNode))
    ]

class Grid:
    def __init__(self):
        self.thresh = None
        self.dt     = None
        self.epoch  = None
        self.dx     = None
        self.xh     = None

class Traj: 
    sigma = None
    b     = None
    r     = None
    T     = None

class Measurement:
    def __init__(self):
        self.mean = None
        self.cov  = None

class BST_ctype(ct.Structure):
    _fields_ = [
        ("root", ct.POINTER(TreeNode)),
        ("dead", ct.POINTER(TreeNode)),
        ("a_count", ct.c_int),
        ("tot_count", ct.c_int),
        ("cfl_min_dt", ct.c_float)
    ]

class BST:
    def __init__(self):
        self.bst = BST_ctype(root = ct.POINTER(TreeNode)(), dead = ct.POINTER(TreeNode)(), a_count = 0, tot_count = 1, cfl_min_dt = sys.float_info.max)
        
    def insert(self, new_node):
        if not self.bst.root:
            self.bst.root = ct.pointer(new_node)
        else:
            self._insert_recursive(self.bst.root, new_node)

    def _insert_recursive(self, node, new_node):
        if new_node.key < node.contents.key:
            if not node.contents.left:
                node.contents.left = ct.pointer(new_node)
            else:
                self._insert_recursive(node.contents.left, new_node)
        elif new_node.key > node.contents.key:
            if not node.contents.right:
                node.contents.right = ct.pointer(new_node)
            else:
                self._insert_recursive(node.contents.right, new_node)

    def search(self, key):
        return bool(self._search_recursive(self.bst.root, key))

    def _search_recursive(self, node, key):
        if not node or node.contents.key == key:
            return node
        if key < node.contents.key:
            return self._search_recursive(node.contents.left, key)
        else:
            return self._search_recursive(node.contents.right, key)
        
    def _min_value_node(self, node):
        current = node
        while(current.contents.left):
            current = current.contents.left
        return current
    
    def delete(self, key):
        if self.bst.root:
            self.bst.root = self._delete_recursive(self.bst.root, key)
    
    def _delete_recursive(self, node, key):
        if not node: 
            return node
        
        if key < node.contents.key:
            node.contents.left = self._delete_recursive(node.contents.left, key)
        elif key > node.contents.key: 
            node.contents.right = self._delete_recursive(node.contents.right, key)
        else:
            if not node.contents.left:
                return node.contents.right
            elif not node.contents.left:
                return node.contents.left
            
            temp = self._min_value_node(node.contents.right)
            node.contents.key = temp.contents.key
            node.contents.right = self._delete_recursive(node.contents.right, temp.contents.key)
        
        return node
         
    def _get_height(self, node):
        if not node:
            return 0 
        
        return 1 + max(self._get_height(node.contents.left), self._get_height(node.contents.right))
    
    def _get_difference(self, node):
        l_height = self._get_height(node.contents.left)
        r_height = self._get_height(node.contents.right)
        return l_height - r_height
    
    def _rr_rotate(self, node):
        t = node.contents.right
        t_copy = TreeNode(key = t.contents.key, left = t.contents.left, right = t.contents.right)
        node.contents.right = t.contents.left
        t_copy.left = node
        return ct.pointer(t_copy)
    
    def _ll_rotate(self, node):
        t = node.contents.left
        t_copy = TreeNode(key = t.contents.key, left = t.contents.left, right = t.contents.right)
        node.contents.left = t.contents.right
        t_copy.right = node
        return ct.pointer(t_copy)
    
    def _lr_rotate(self, node):
        t = node.contents.left
        node.contents.left = self._rr_rotate(t)
        return self._ll_rotate(node)
    
    def  _rl_rotate(self, node):
        t = node.contents.right
        node.contents.right = self._ll_rotate(t)
        return self._rr_rotate(node)
    
    def balance(self):
        node = self.bst.root
        bal_factor = self._get_difference(node)
        while(abs(bal_factor) > 1):
            if bal_factor > 1:
                if self._get_difference(node.contents.left) > 0:
                    node = self._ll_rotate(node)
                else:
                    node = self._lr_rotate(node)
            elif bal_factor < -1:
                if self._get_difference(node.contents.right) > 0:
                    node = self._rl_rotate(node)
                else:
                    node = self._rr_rotate(node)
            bal_factor = self._get_difference(node)
        self.bst.root = node

    def print_tree(self):
        if self.bst.root:
            self._print_tree_recursive(self.bst.root, "", True)
            print("\n")
        else:
            print("BST is empty")
   
    def _print_tree_recursive(self, node, indent, last):
        if node:
            print(indent, end="")
            if indent == "":
                print("ROOT----", end="")
                indent += "        "
            else:
                if last:
                    print("R----", end="")
                    indent += "     "
                else:
                    print("L----", end="")
                    indent += "|    "
            print(node.contents.key)
            self._print_tree_recursive(node.contents.left, indent, False)
            self._print_tree_recursive(node.contents.right, indent, True)

In [3]:
def rosenberg_pair(state, d, m):
    if d==1:
        return state[0]
    
    new_state = np.empty(d-1)
    for i in range(d-1):
        new_state[i] = state[i]

    new_m = np.max(new_state)
    return rosenberg_pair(new_state, d-1, new_m) + m**d + (m - state[d-1])*((m + 1)**(d - 1) - m**(d - 1))

def state_conversion(state, DIM):
    shift_state = np.empty(DIM)
    for i in range(DIM):
        if(state[i] < 0):
            shift_state[i] = -2*state[i] - 1
        else:
            shift_state[i] = 2*state[i]

    m = np.max(shift_state)
    return rosenberg_pair(shift_state, DIM, m)

def mc(th):
    return max(0, min((1 + th)/2, 2.0, 2*th))

def get_size(root):
    if root == None:
        return 0
    
    return 1 + get_size(root.left) + get_size(root.right)

In [17]:
import random

root = TreeNode(key=4)
P = BST()
P.insert(root)

for i in range(3):
    P.insert(TreeNode(key=3-i))
    P.insert(TreeNode(key=i+5))

P.print_tree()

P.balance()

P.print_tree()

ROOT----4
        L----3
        |    L----2
        |    |    L----1
        R----5
             R----6
                  R----7


ROOT----4
        L----3
        |    L----2
        |    |    L----1
        R----5
             R----6
                  R----7




In [4]:
#========================================== Begin User Input ===============================================#
print( "Reading in user inputs...\n" )

NM        = 1          # Number of measurements 
FILE_PATH = "./Epochs" # Measurement file path

G           = Grid()   # Grid instance
G.thresh    = 2E-5     # Probability threshold
OUTPUT      = True     # Write info to terminal
RECORD      = True     # Write PDFs to .txt file
MEASURE     = True     # Take discrete measurement updates
OUTPUT_FREQ = 20       # Number of steps per output to terminal
DEL_STEP    = 25       # Number of steps per deletion procedure
NUM_DIST    = 6        # Number of distributions recorded per measurement
#===========================================================================================================#

Reading in user inputs...



In [5]:
#===================================== Read in measurement/trajectory info =================================#
print("Reading in discrete measurements...\n")

measurement_file = open(FILE_PATH + "/measurements.txt", "r")
mean = np.empty(DIM)
cov  = np.empty((DIM,DIM))

measurement_file.readline(); # Skip label line
mean = [float(string) for string in measurement_file.readline().split(" ")] # Mean
measurement_file.readline(); # Skip blank space
measurement_file.readline(); # Skip label line
for i in range(DIM):
    cov[i] = [float(string) for string in measurement_file.readline().split(" ")] # Covariance
G.epoch = mean
G.dx = [x/2 for x in np.diag(cov)] # Grid width
G.xh = [x/2 for x in G.dx]         # Half grid width

Lor = Traj() 
measurement_file.readline(); # Skip blank space
measurement_file.readline(); # Skip label line
Lor.sigma = float(measurement_file.readline())
measurement_file.readline(); # Skip label line
Lor.b = float(measurement_file.readline())
measurement_file.readline(); # Skip label line
Lor.r = float(measurement_file.readline())
measurement_file.readline(); # Skip label line
Lor.T = float(measurement_file.readline())

measure_time = Lor.T/NM                 # Time between measurements
record_time = measure_time/(NUM_DIST-1) # Time between recording PDF
measurement_file.close()
#===========================================================================================================#

Reading in discrete measurements...



In [18]:
# Python3 program to convert a left
# unbalanced BST to a balanced BST
import sys
import math

# A binary tree node has data, pointer to left child
# and a pointer to right child
class Node:
    def __init__(self,data):
        self.data=data
        self.left=None
        self.right=None

# This function traverse the skewed binary tree and
# stores its nodes pointers in vector nodes[]
def storeBSTNodes(root,nodes):

    # Base case
    if not root:
        return

    # Store nodes in Inorder (which is sorted
    # order for BST)
    storeBSTNodes(root.left,nodes)
    nodes.append(root)
    storeBSTNodes(root.right,nodes)

# Recursive function to construct binary tree
def buildTreeUtil(nodes,start,end):

    # base case
    if start>end:
        return None

    # Get the middle element and make it root
    mid=(start+end)//2
    node=nodes[mid]

    # Using index in Inorder traversal, construct
    # left and right subtress
    node.left=buildTreeUtil(nodes,start,mid-1)
    node.right=buildTreeUtil(nodes,mid+1,end)
    return node

# This functions converts an unbalanced BST to
# a balanced BST
def buildTree(root):

    # Store nodes of given BST in sorted order
    nodes=[]
    storeBSTNodes(root,nodes)

    # Constructs BST from nodes[]
    n=len(nodes)
    return buildTreeUtil(nodes,0,n-1)

# Function to do preorder traversal of tree
def preOrder(root):
    if not root:
        return  
    print("{} ".format(root.data),end="")
    preOrder(root.left)
    preOrder(root.right)

# Driver code
if __name__=='__main__':
    # Constructed skewed binary tree is
    # 10
    # /
    # 8
    # /
    # 7
    # /
    # 6
    # /
    # 5
    root = Node(10)
    root.left = Node(8)
    root.left.left = Node(7)
    root.left.left.left = Node(6)
    root.left.left.left.left = Node(5)
    root = buildTree(root)
    print("Preorder traversal of balanced BST is :")
    preOrder(root)

Preorder traversal of balanced BST is :
7 5 6 8 10 